# Breast cancer predictor (regression) using DNN

## Step 1: Prepare training and evaluation dataset, create FastEstimator Pipeline

Pipeline can take both data in memory and data in disk. In this example, we are going to use data in memory by loading data with sklearn.datasets.load_breast_cancer.
The following can be used to get the description of the data:

In [1]:
from sklearn.datasets import load_breast_cancer
import pandas as pd

breast_cancer = load_breast_cancer()
print(breast_cancer.DESCR)
print(pd.DataFrame(breast_cancer.data).head())

ModuleNotFoundError: No module named 'sklearn'

Next we need to scale the inputs to the neural network. This is done by using a StandardScaler.

In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_eval = scaler.transform(x_eval)

For in-memory data in Pipeline, the data format should be a nested dictionary like: {"mode1": {"feature1": numpy_array, "feature2": numpy_array, ...}, ...}. Each mode can be either train or eval, in our case, we have both train and eval. feature is the feature name, in our case, we have x and y. The network prediction will be a rank-1 array, in order to match prediction, we will expand the groud truth dimension by 1.

In [4]:
import numpy as np

train_data = {"x": x_train, "y": np.expand_dims(y_train, -1)}
eval_data = {"x": x_eval, "y": np.expand_dims(y_eval, -1)}
data = {"train": train_data, "eval": eval_data}

In [5]:
#Parameters
epochs = 50
batch_size = 32
steps_per_epoch = None
validation_steps = None

Now we are ready to define Pipeline:

In [6]:
import fastestimator as fe

pipeline = fe.Pipeline(batch_size=batch_size, data=data)

## Step 2: Prepare model, create FastEstimator Network

First, we have to define the network architecture in tf.keras.Model or tf.keras.Sequential. After defining the architecture, users are expected to feed the architecture definition and its associated model name, optimizer and loss name (default to be 'loss') to FEModel.

In [7]:
from tensorflow.keras import layers

def create_dnn():
    model = tf.keras.Sequential()

    model.add(layers.Dense(32, activation="relu", input_shape=(30, )))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(16, activation="relu"))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(8, activation="relu"))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation="linear"))

    return model

model = fe.build(model_def=create_dnn, model_name="dnn", optimizer="adam", loss_name="loss")

Now we are ready to define the Network: given with a batch data with key x and y, we have to work our way to loss with series of operators. ModelOp is an operator that contains a model.

In [8]:
from fastestimator.op.tensorop import ModelOp, MeanSquaredError

network = fe.Network(
    ops=[ModelOp(inputs="x", model=model, outputs="y_pred"), MeanSquaredError(inputs=("y","y_pred"),outputs="loss")])

## Step 3: Configure training, create Estimator

During the training loop, we want to: 1) measure lowest loss for data data 2) save the model with lowest valdiation loss. Trace class is used for anything related to training loop, we will need to import the ModelSaver trace.

In [9]:
import tempfile
from fastestimator.trace import ModelSaver

model_dir = tempfile.mkdtemp()
traces = [ModelSaver(model_name="dnn", save_dir=model_dir, save_best=True)]

Now we can define the Estimator and specify the training configuation:

In [10]:
estimator = fe.Estimator(network=network, 
                         pipeline=pipeline, 
                         epochs=epochs, 
                         steps_per_epoch=steps_per_epoch,
                         validation_steps=validation_steps,
                         traces=traces)

## Start Training

In [ ]:
estimator.fit()

## Inferencing

After training, the model is saved to a temporary folder. we can load the model from file and do inferencing on a sample.

In [12]:
import os

model_path = os.path.join(model_dir, 'dnn_best_loss.h5')
trained_model = tf.keras.models.load_model(model_path, compile=False)

Randomly get one sample from validation set and compare the predicted value with model's prediction:

In [13]:
import numpy as np

selected_idx = np.random.randint(0, high=101)
print("test sample idx {}, ground truth: {}".format(selected_idx, y_eval[selected_idx]))

test_sample = np.expand_dims(x_eval[selected_idx], axis=0)

predicted_value = trained_model.predict(test_sample)
print("model predicted value is {}".format(predicted_value))

test sample idx 2, ground truth: 19.0
model predicted value is [[20.183064]]
